In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pybatfish.client.session import Session
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Set up a connection to the batfish session
bf = Session(host="localhost")

In [8]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "toy_network"
SNAPSHOT_NAME = "simple_snapshot"

SNAPSHOT_PATH = "scenarios/toy/ospf/"

# Now create the network and initialize the snapshot
bf.set_network(NETWORK_NAME)
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

# Not really sure this is the way to go about including the BGP configurations
#BGP_PATH = "scenarios/bics/bgp/"
#bf.fork_snapshot(SNAPSHOT_NAME, name=SNAPSHOT_NAME+"_with_bgp", overwrite=True, add_files=BGP_PATH)

'simple_snapshot'

In [9]:
bf.set_snapshot(SNAPSHOT_NAME)

# Get routing tables for all nodes and VRFs
routes_all = bf.q.routes().answer().frame()

filtered_df = routes_all[(routes_all['Next_Hop_Interface'] == 'FastEthernet0/0') & (routes_all['Node'] == 'a')]
filtered_df

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
9,a,default,100.0.0.0/24,interface FastEthernet0/0,AUTO/NONE(-1l),FastEthernet0/0,connected,0,0,None
10,a,default,100.0.0.1/32,interface FastEthernet0/0,AUTO/NONE(-1l),FastEthernet0/0,local,0,0,None


In [10]:
# Run a comparison between the two FIBs
ms_fib = pd.DataFrame(columns=['Node','VRF','Network','Next_Hop_Interface','Protocol'])

# Open the FIB file for reading
with open('scenarios/bics/ospf/fibs/fib-1.txt', 'r') as file:
    router = None
    ROUTER_IDENTIFIER = "# Router:"
    vrf = None
    VRF_IDENTIFIER = "## VRF:"

    protocol_aliases = {"ConnectedRoute": "connected",
                        "OspfIntraAreaRoute": "ospf",
                        "OspfExternalType2Route": "ospfE2"
                        }

    for line in file:
        # Process each line
        if ROUTER_IDENTIFIER in line: 
            router = line.replace(ROUTER_IDENTIFIER,"")[:-1]
        elif VRF_IDENTIFIER in line: 
            vrf = line.replace(VRF_IDENTIFIER,"")[:-1]
        else:
            entry_info = line.split(';')

            data = {'Node': router,
                    'VRF': vrf,
                    'Network': entry_info[0],
                    'Next_Hop_Interface': entry_info[1],
                    'Protocol': protocol_aliases[entry_info[2][:-1]]
                }
            ms_fib = ms_fib.append(data, ignore_index=True)

def bf_to_fib(routes, filename):
    # Filter out local routes
    routes = routes[routes.Protocol != "local"]
    # Sort the routes by nodes and VRFs
    sorted_routes = routes.sort_values(by=['Node', 'VRF'])

    router = None
    ROUTER_IDENTIFIER = "# Router:{}\n"
    vrf = None
    VRF_IDENTIFIER = "## VRF:{}\n"

    inverse_protocol_aliases = {"connected": "ConnectedRoute",
                                "ospf": "OspfIntraAreaRoute",
                                "ospfE2": "OspfExternalType2Route"
                                }

    with open(SNAPSHOT_PATH+"tmp_fibs/"+filename, 'w') as file:
        for index, row in sorted_routes.iterrows():

            if router != row["Node"]:
                # We have a new router, start a new router
                router = row["Node"]
                file.write(ROUTER_IDENTIFIER.format(router))
                # Reset the VRF
                vrf = None
            if vrf != row["VRF"]:
                # We have a new VRF, start a new VRF
                vrf = row["VRF"]
                file.write(VRF_IDENTIFIER.format(vrf))
            # Write a line to the config
            file.write(row["Network"]+";"+row["Next_Hop_Interface"]+";"+inverse_protocol_aliases[row["Protocol"]]+"\n")

bf_to_fib(routes_all, "fib-test.txt")

# Quick sanity check
def check_lines(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines1 = set(f1.read().splitlines())
        lines2 = set(f2.read().splitlines())

        # Check if all lines in file1 are in file2
        return lines1.issubset(lines2)

print(check_lines("scenarios/bics/ospf/fibs/fib-1.txt", "scenarios/bics/ospf/tmp_fibs/fib-test.txt"))

ms_fib

True


,Node,VRF,Network,Next_Hop_Interface,Protocol
0,amsterdam,default,10.0.0.32/31,FastEthernet2/0,connected
1,amsterdam,default,10.0.0.34/31,FastEthernet2/0,ospf
2,amsterdam,default,100.0.9.0/24,FastEthernet2/0,ospfE2
3,amsterdam,default,100.0.18.0/24,FastEthernet1/1,ospfE2
4,amsterdam,default,10.0.0.66/31,FastEthernet1/1,ospf
...,...,...,...,...,...
3088,zurich,default,10.0.0.54/31,FastEthernet0/1,ospf
3089,zurich,default,100.0.15.0/24,FastEthernet0/1,ospfE2
3090,zurich,default,100.0.32.0/24,FastEthernet0/1,ospfE2
3091,zurich,default,10.0.0.60/31,FastEthernet0/1,ospf


In [11]:
bf_fib = routes_all[['Node','VRF','Network','Next_Hop_Interface','Protocol']]

df1_comp = ms_fib.sort_values(by = ['Node','VRF','Network','Next_Hop_Interface','Protocol']).reset_index(drop = True)
df2_comp = bf_fib.sort_values(by = ['Node','VRF','Network','Next_Hop_Interface','Protocol']).reset_index(drop = True)

combined_df = pd.concat([df1_comp, df2_comp])

difference_df = combined_df.drop_duplicates(keep=False)

difference_df

,Node,VRF,Network,Next_Hop_Interface,Protocol
0,amsterdam,default,10.0.0.0/31,FastEthernet0/1,ospf
1,amsterdam,default,10.0.0.0/31,FastEthernet1/1,ospf
2,amsterdam,default,10.0.0.10/31,FastEthernet1/0,ospf
3,amsterdam,default,10.0.0.12/31,FastEthernet1/0,ospf
4,amsterdam,default,10.0.0.14/31,FastEthernet0/1,ospf
...,...,...,...,...,...
51,d,default,100.0.0.0/24,FastEthernet2/0,ospfE2
52,d,default,100.0.10.0/24,FastEthernet3/1,ospfE2
53,d,default,100.0.20.0/24,FastEthernet3/1,ospfE2
54,d,default,100.0.30.0/24,FastEthernet0/0,connected


In [12]:
unique_values = difference_df['Protocol'].nunique()
print("Number of unique values:", unique_values)

Number of unique values: 4


In [13]:
result = bf.q.layer3Edges().answer().frame()

bf_topology = set()

for index, row in result.iterrows():
    interface_tuple = (str(row["Interface"]).replace("[", ":")[:-1],str(row["Remote_Interface"]).replace("[", ":")[:-1])
    bf_topology.add(interface_tuple)

ms_topology = set()
# Open and read the text file
with open('scenarios/bics/ospf/topology.txt', 'r') as file:
    for line in file:
        # Remove angle brackets and split the line by ","
        line = line.strip("<>\n").strip()  # Remove angle brackets and leading/trailing spaces
        parts = line.split(",")

        # Create a tuple from the parts and add it to the set
        if len(parts) == 2:
            ms_topology.add((parts[0].strip(), parts[1].strip()))

# Substring search
query_result = result[result['Interface'].astype(str).str.contains("a", na=False)]
query_result

,Interface,IPs,Remote_Interface,Remote_IPs
0,a[FastEthernet1/0],['10.0.0.5'],d[FastEthernet2/0],['10.0.0.4']
1,a[FastEthernet2/0],['10.0.0.7'],b[FastEthernet1/1],['10.0.0.6']
2,b[FastEthernet1/1],['10.0.0.6'],a[FastEthernet2/0],['10.0.0.7']
3,b[FastEthernet2/1],['10.0.0.3'],d[FastEthernet3/1],['10.0.0.2']
4,b[FastEthernet3/1],['10.0.0.9'],c[FastEthernet2/1],['10.0.0.8']
5,c[FastEthernet2/1],['10.0.0.8'],b[FastEthernet3/1],['10.0.0.9']
6,d[FastEthernet2/0],['10.0.0.4'],a[FastEthernet1/0],['10.0.0.5']
7,d[FastEthernet3/1],['10.0.0.2'],b[FastEthernet2/1],['10.0.0.3']


In [30]:
from helper import get_policy_db
from helper import get_sampler
from helper import init_dp_engine
from collections import defaultdict

from config2spec.topology.links import Link
from config2spec.topology.links import LinkState
from config2spec.topology.topology import NetworkTopology
from config2spec.topology.interface import Interface
from config2spec.netenv.network_environment import NetworkEnvironment
from config2spec.policies.policy_db import PolicyStatus

import random

# Function to build a topology from the batfish result
def build_topology():
    topology = NetworkTopology()

    # Extract router information from the batfish routing table
    routers = set(routes_all['Node'].unique())
    # Extract edge information from the batfish link information
    edges = set()
    for index, edge in result.iterrows():
        first_interface = str(edge['Interface']).split('[')
        second_interface = str(edge['Remote_Interface']).split('[')
        edges.add((first_interface[0],first_interface[1][:-1],second_interface[0],second_interface[1][:-1]))

    for i, router in enumerate(routers):
        router_id = "r%d" % i

        # Extract interface information from the topology datastructure and the route information
        interfaces = defaultdict(dict)
        # TODO: The "FastEthernet0/0" interface is always hardcoded, get the IP from route information
        intf_name = "FastEthernet0/0"
        interface = Interface(intf_name)
        query = routes_all[(routes_all['Next_Hop_Interface'] == 'FastEthernet0/0') & (routes_all['Node'] == router) & (routes_all['Protocol'] == 'local')]
        ip_address = str(query["Network"].iloc[0]).split('/')[0]
        if intf_name not in interfaces: interfaces[intf_name] = interface
        interfaces[intf_name].set_ip_address(ip_address, "{}/24".format(ip_address))
        
        # Query for all other router interfaces
        query = result[result['Interface'].astype(str).str.contains(router, na=False)]
        for index, ans in query.iterrows():
            # Set the interface name
            intf_name = str(ans["Interface"]).split("[")[1][:-1]
            interface = Interface(intf_name)
            # Set the IP
            # TODO: We hardcode the subnet, maybe read from the FIBs?
            ip_address = str(ans["IPs"]).strip('[]').replace("'", "")
            # Add interface
            if intf_name not in interfaces: interfaces[intf_name] = interface
            interfaces[intf_name].set_ip_address(ip_address, "{}/31".format(ip_address))

        # Just set the ACLs to the empty set
        # TODO: Violently incorrect, just a placeholder for now
        access_lists = set()
        topology.add_router(router, router_id, interfaces=interfaces, access_lists=access_lists)
    
    # LINKS maybe use the edge information
    for r1, r1_intf, r2, r2_intf in edges:
        topology.add_link(r1, r2, 1)
        topology.next_hops[r1][r1_intf] = r2
        # Add interface, this is not done in the original code
        topology.set_interface(r1,r2,r1_intf)
     
    return topology


# In order to figure out if the ranking is actually accurate, we should first initialise the same general settings
seed = 8006
window_size = 10
sampling_mode = "sum"
trimming = True
waypoints_min = 3
waypoints_fraction = 5
max_failures = 1


# This snippet does what the `build_network` function does
# ----------------------------------------------------------------------------------------------
random.seed(seed)

network = build_topology()

# get waypoints, sort them in order to get a consistent ordering for the random sampling
all_routers = sorted(network.nodes())
num_waypoints = max(waypoints_min, int(len(all_routers) / waypoints_fraction))
waypoints = random.sample(all_routers, num_waypoints)

waypoints = []

links = list()
all_edges = network.get_undirected_edges()
for i, edge in enumerate(all_edges):
    links.append(Link("l{id}".format(id=i), edge, LinkState.SYMBOLIC))

# create the network environment
netenv = NetworkEnvironment(links, k_failures=max_failures)
# ----------------------------------------------------------------------------------------------
dp_engine = init_dp_engine(network, SNAPSHOT_PATH+"tmp_fibs/", debug=False)

# init policy Database
policy_db = get_policy_db(network, waypoints=waypoints)

# get sampler
sampler = get_sampler(sampling_mode, netenv, policy_db, seed)

In [31]:
import pickle
from ipaddress import IPv4Network
from config2spec.topology.links import Link
from pybatfish.datamodel.primitives import Interface


first = True
prev_forwarding_graphs = False
step = 1

data = {"failed_links": [],
        "policies": []
        }

# Now run the main loop
while True:
    remaining_samples = sampler.remaining_samples()
    print("We have {} remaining samples".format(remaining_samples))
    remaining_policies = policy_db.num_policies(status=PolicyStatus.UNKNOWN)

    # pick a concrete environment
    if first:
        concrete_env = sampler.get_all_up()
    else:        
        # check if we are done
        if remaining_samples == 0 or remaining_policies == 0:
            break

        concrete_env = sampler.get_next_env(prev_forwarding_graphs)

    # compute the dataplane and check the policies that hold
    num_eliminated_policies = -1
    failed_edges = concrete_env.get_links(state=LinkState.DOWN)

    # Fork a snapshot with the link failures
    FAIL_SNAPSHOT_NAME = "fail_snapshot"
    bf.fork_snapshot(
        SNAPSHOT_NAME,
        FAIL_SNAPSHOT_NAME,
        deactivate_interfaces=[Interface(link.edge[0],network.get_interface(link.edge[0],link.edge[1])) for link in failed_edges],
        overwrite=True)
    # Get bf routing tables
    bf.set_snapshot(FAIL_SNAPSHOT_NAME)
    routes = bf.q.routes().answer().frame()
    # Extract FIBs from those
    fib_file_name = "fib-{}.txt".format(step)
    bf_to_fib(routes, fib_file_name)
    
    forwarding_graphs = dp_engine.get_forwarding_graphs(fib_file_name)
    dominator_graphs = dp_engine.get_dominator_graphs()

    if first:
        prev_forwarding_graphs = forwarding_graphs
        policy_db.update_policies(9, forwarding_graphs, dominator_graphs)
        first = False

    policies = policy_db.policy_guesser.get_policies(forwarding_graphs, dominator_graphs)
    print("We have {} policies holding for this case\n".format(len(policies)))

    """
    change, guess_size = policy_db.update_policies(9, forwarding_graphs, dominator_graphs)

        prev_forwarding_graphs = forwarding_graphs
        if prev_guess_size >= 0:
            num_eliminated_policies = prev_guess_size - guess_size
        prev_guess_size = guess_size
    """
    
    # Save data
    data["policies"].append(policies)
    data["failed_links"].append(failed_edges)
    step += 1

with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)

We have 5 remaining samples


KeyError: 'a'